* Combine best N feateures from Uni+Bigram+Negation
* together with w2v and glove feature
* apply SVM/LogRegress(N feateures+w2v+glove)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import pandas as pd
import pickle as pk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import precision_score, recall_score,f1_score
from gensim.models.word2vec import Word2Vec 

In [3]:
train_path = "/content/drive/MyDrive/colab/nus/5004 PLP/nmsm/datasets/Data/trainset.pk"
test_path = "/content/drive/MyDrive/colab/nus/5004 PLP/nmsm/datasets/Data/testset.pk"

trainset = pk.load(open(train_path, "rb"))
testset = pk.load(open(test_path, "rb"))

In [4]:
def nega_tag(text):
    transformed = re.sub(r"\b(?:never|nothing|nowhere|noone|none|not|haven't|hasn't|hasnt|hadn't|hadnt|can't|cant|couldn't|couldnt|shouldn't|shouldnt|won't|wont|wouldn't|wouldnt|don't|dont|doesn't|doesnt|didn't|didnt|isnt|isn't|aren't|arent|aint|ain't|hardly|seldom)\b[\w\s]+[^\w\s]", lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), text, flags=re.IGNORECASE)
    return(transformed)

In [5]:
train_set_nega = []
for doc in trainset:
    trans = nega_tag(doc[0])
    lab = doc[1]
    train_set_nega.append([trans, lab])

In [6]:
X_nega_train = [t[0] for t in train_set_nega]
Y_nega_train = [t[1] for t in train_set_nega]

In [9]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
train_nega_vectors = vectorizer.fit_transform(X_nega_train)

In [11]:
ch21 = SelectKBest(chi2, k=100)
train_Kbest = ch21.fit_transform(train_nega_vectors, Y_nega_train)
train_Kbest_new=train_Kbest.toarray()

In [14]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
            #self.dim=len(word2vec["and"])
        else:
            self.dim=0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [19]:
X_train = [t[0].split() for t in trainset]
X_test = [t[0].split() for t in testset]
X = X_train + X_test

Y_train = [t[1] for t in trainset]
Y_test = [t[1] for t in testset]
y = Y_train + Y_test

In [20]:
model = Word2Vec(X, size=100, window=5, min_count=2, workers=2)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

<ipython-input-20-4c9c9ac8dbda>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}


In [ ]:
glove_small = {}

In [21]:
w2v_Embedding = MeanEmbeddingVectorizer(w2v)
glove_Embedding = MeanEmbeddingVectorizer(glove_small)

NameError: ignored